In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.2 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=00f3d1e844d306a2a134d834e68c30edfa59531b8755736d01b64e653e260e30
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
import pandas as pd
import torchtext
import numpy as np
import torch
import torch.nn as nn
import random
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
zip_path = "/content/drive/MyDrive/aksharantar_sampled.zip"
!cp "{zip_path}" .
!unzip -q aksharantar_sampled.zip
!rm aksharantar_sampled.zip 

In [ ]:
def get_data(lang):
  train_csv=f"aksharantar_sampled/{lang}/{lang}_train.csv"
  test_csv=f"aksharantar_sampled/{lang}/{lang}_test.csv"
  val_csv=f"aksharantar_sampled/{lang}/{lang}_valid.csv"
  
  return train_csv,test_csv,val_csv


In [ ]:
train,test,val=get_data("hin")

In [ ]:
### For train
input_texts = []
target_texts = []

train_df = pd.read_csv(train, header=None, names=["1", "2"]).astype(str)

# Add all the input and target texts with start sequence and end sequence added to target 
for index, row in train_df.iterrows():
      input_text = row['1']
      target_text = row['2']
      if target_text == '' or input_text == '':
          continue
      target_text = "\t" + target_text + "\n"
      input_texts.append(input_text)
      target_texts.append(target_text)

english_tokens = set()
hindi_tokens = set()

for x,y in zip(input_texts,target_texts):
    for ch in x:
        english_tokens.add(ch)
    for ch in y:
        hindi_tokens.add(ch)
    
english_tokens = sorted(list(english_tokens))
hindi_tokens = sorted(list(hindi_tokens))

eng_token_map = dict([(ch,i+1) for i,ch in enumerate(english_tokens)])
hin_token_map = dict([(ch,i+1) for i,ch in enumerate(hindi_tokens)])

eng_token_map["<UNK>"]=len(english_tokens)+1
hin_token_map["<UNK>"]=len(hindi_tokens)+1
eng_token_map['<PAD>']=0
hin_token_map['<PAD>']=0

max_eng_len = max([len(i) for i in input_texts])
max_hin_len = max([len(i) for i in target_texts])

In [ ]:
def pre_process(data):
    
    input_texts = []
    target_texts = []
    
    df = pd.read_csv(data, header=None, names=["1", "2"]).astype(str)

    for index, row in df.iterrows():
      input_text = row['1']
      target_text = row['2']
      if target_text == '' or input_text == '':
          continue
      target_text = "\t" + target_text + "\n"
      input_texts.append(input_text)
      target_texts.append(target_text)

    
    a = np.zeros((len(input_texts),max_eng_len+2),dtype="float32")
    b = np.zeros((len(target_texts),max_hin_len+2),dtype="float32")
    
    
    for i,(x,y) in enumerate(zip(input_texts,target_texts)):
        for j,ch in enumerate(x):
            a[i,j] = eng_token_map.get(ch,eng_token_map["<UNK>"])

        for j,ch in enumerate(y):
            b[i,j] = hin_token_map.get(ch,hin_token_map["<UNK>"])
        
      
    return a,b

In [ ]:
trainx, trainy= pre_process(train)
valx, valy= pre_process(val)
testx,testy,= pre_process(test)

In [ ]:
reverse_eng_map = dict([(i,char) for char,i in eng_token_map.items()])
reverse_hin_map = dict([(i,char) for char,i in hin_token_map.items()])

In [ ]:

class CustomDataset(Dataset):
    def __init__(self, input_data, target_data):
        self.input_data = input_data
        self.target_data = target_data
    

    def __len__(self):
        return len(self.input_data)

    def __getitem__(self, idx):
        input_seq = self.input_data[idx]
        target_seq = self.target_data[idx]

        return input_seq, target_seq

def custom_collate(batch):
    input_seqs, target_seqs= zip(*batch)
    input_seqs = torch.from_numpy(np.stack(input_seqs, axis=1))
    target_seqs = torch.from_numpy(np.stack(target_seqs, axis=1))

    return input_seqs, target_seqs


train_dataset = CustomDataset(trainx, trainy)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=custom_collate)

val_dataset = CustomDataset(valx, valy)
#val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False, collate_fn=custom_collate)

test_dataset = CustomDataset(testx, testy)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=custom_collate)



In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'


class Encoder(nn.Module):
    def __init__(self, input_size, embed_dim, hidden_size, num_layers, dropout,cell_type):
        super().__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, embed_dim,padding_idx=0)
        self.dropout = nn.Dropout(dropout)
        self.cell_type=cell_type
        self.fc=nn.Linear(hidden_size*2,hidden_size) # To connect last layer of encoder to first layer of decoder, used when bidirectional=True
                                                     # The hidden_size  of last enocder layer and 1st layer of decoder are same in general
        if cell_type=="LSTM":
            self.rnn=nn.LSTM(embed_dim, hidden_size, num_layers, dropout=dropout,bidirectional=True)
            self.cell=nn.Linear(hidden_size*2,hidden_size)
        elif cell_type=="GRU":
            self.rnn=nn.GRU(embed_dim, hidden_size, num_layers, dropout=dropout,bidirectional=True)
        else:
            self.rnn=nn.RNN(embed_dim, hidden_size, num_layers, dropout=dropout,bidirectional=True)
    
    def forward(self, x):
        embedded = self.dropout(self.embedding(x))

        if self.cell_type=="LSTM":
            output, (hidden, cell) = self.rnn(embedded)
            concated = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
            hidden = torch.tanh(self.fc(concated))

            cell_concat=torch.cat((cell[-2,:,:], cell[-1,:,:]), dim=1)
            cell=torch.tanh(self.fc(cell_concat))


            return output,hidden,cell
        
        elif self.cell_type=="GRU":
            output, hidden = self.rnn(embedded)
            concated = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
            hidden = torch.tanh(self.fc(concated))

            return output, hidden
        
        else:
            output, hidden = self.rnn(embedded)
            concated = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
            hidden = torch.tanh(self.fc(concated))

            return output,hidden

class Attention(nn.Module):
    def __init__(self,hidden_size):
        super().__init__()
        self.enc_hidden_size=hidden_size
        self.dec_hidden_size=hidden_size

        self.fc1 = nn.Linear(self.enc_hidden_size * 2 + self.dec_hidden_size, self.dec_hidden_size)
        self.fc2 = nn.Linear(self.dec_hidden_size, 1, bias=False)
    
    def forward(self,encoder_output,hidden):
        
        batch_size = encoder_output.shape[1]
        source_len = encoder_output.shape[0]
        
        hidden = hidden.unsqueeze(1).repeat(1, source_len, 1)
        
        output = encoder_output.permute(1, 0, 2)
        

        energy = torch.tanh(self.fc1(torch.cat((hidden, output), dim = 2))) 
        

        attention = self.fc2(energy).squeeze(dim=2)
        
        attention_weight = torch.softmax(attention, dim=1)
        
        return attention_weight



class Decoder(nn.Module):
    def __init__(self, output_size, embed_dim, hidden_size, num_layers, dropout,cell_type,attention):
        super().__init__()
        self.hidden_size = hidden_size
        self.cell_type=cell_type
        self.embedding = nn.Embedding(output_size, embed_dim,padding_idx=0)
        if cell_type=="LSTM":
          self.rnn = nn.LSTM((hidden_size*2)+embed_dim, hidden_size, num_layers,  dropout=dropout)
        elif cell_type=="GRU":
          self.rnn=nn.GRU((hidden_size*2)+embed_dim,hidden_size,num_layers,dropout=dropout,bidirectional=False)
        else:
          self.rnn=nn.RNN(hidden_size*2+embed_dim,hidden_size,num_layers,dropout=dropout)

        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout=nn.Dropout(dropout)
        self.attention=attention

    def forward(self, x,encoder_output,hidden,cell):
        attention = self.attention(encoder_output, hidden).unsqueeze(1)
        output = encoder_output.permute(1, 0, 2)
        context = torch.bmm(attention, output).permute(1, 0, 2)
        x = x.unsqueeze(0)
        embedded = self.dropout(self.embedding(x))

        rnn_input = torch.cat((embedded, context), dim=2)
        if self.cell_type=="LSTM":
          output,(hidden,cell) = self.rnn(rnn_input,(hidden.unsqueeze(0),cell.unsqueeze(0)))
          output= self.fc(output.squeeze(0))
          return output, hidden.squeeze(0),cell.squeeze(0), attention
        
        elif self.cell_type=="GRU":
          output,hidden = self.rnn(rnn_input,hidden.unsqueeze(0))
          output = output.squeeze(0)
          output= self.fc(output)
          return output, hidden.squeeze(0)
        
        else:
          output,hidden = self.rnn(rnn_input,hidden.unsqueeze(0))
          output = output.squeeze(0)
          output= self.fc(output)
          return output, hidden.squeeze(0)

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    
       
    def forward(self, source, target, teacher_forcing_ratio=0.5):
        batch_size = source.shape[1]
        target_len = target.shape[0]
        target_vocab_size = len(hin_token_map)

        outputs = torch.zeros(target_len,batch_size, target_vocab_size).to(device)
        if self.encoder.cell_type=="LSTM":
          enc_output ,hidden, cell = self.encoder(source)

          x = target[0]

          for t in range(1, target_len):
              output, hidden, cell,att = self.decoder(x,enc_output, hidden, cell)
              outputs[t] = output
              top1 = output.argmax(1)
              if random.random() < teacher_forcing_ratio:
                  x = target[t]
              else:
                  x = top1

          return outputs, att
        
        elif self.encoder.cell_type=="GRU":
          enc_output,hidden = self.encoder(source)

          x = target[0]

          for t in range(1, target_len):
              output,hidden=self.decoder(x,enc_output,hidden,None)
              outputs[t] = output
              top1= output.argmax(1)
              if random.random() < teacher_forcing_ratio:
                  x = target[t]
              else:
                  x = top1
          return outputs
        
        else:
          enc_output,hidden = self.encoder(source)

          x = target[0]

          for t in range(1, target_len):
              output,hidden=self.decoder(x,enc_output,hidden,None)
              outputs[t] = output
              top1= output.argmax(1)
              if random.random() < teacher_forcing_ratio:
                  x = target[t]
              else:
                  x = top1
          return outputs


def build_model(cell = "LSTM",nunits = 64, enc_dec_layers = 1,embed_dim = 128,dropout=0):
    att=Attention(nunits)
    encoder = Encoder(input_size=len(eng_token_map), embed_dim=embed_dim, hidden_size=nunits, num_layers=enc_dec_layers, dropout=dropout,cell_type=cell)
    decoder = Decoder(output_size=len(hin_token_map), embed_dim=embed_dim, hidden_size=nunits, num_layers=enc_dec_layers, dropout=dropout,cell_type=cell,attention=att)
    model = Seq2Seq(encoder, decoder)
    return model

In [ ]:

def train(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0.0
    total_chars = 0
    correct_chars = 0

    for i, (input_seq, target_seq) in enumerate(dataloader):
        input_seq = input_seq.long().to(device)
        target_seq = target_seq.long().to(device)

        optimizer.zero_grad()

        output,att_wt = model(input_seq, target_seq)
        _, predicted = torch.max(output, dim=2)


        for j in range(predicted.shape[1]):
              predicted_seq = predicted[:, j]
              targets_seq = target_seq[:, j]

              # Find the index of the first EOS token in the sequence (for character & word-level accuracy)
              eos_idx = (targets_seq == hin_token_map["\n"]).nonzero()
              if eos_idx.numel() > 0:
                  eos_idx = eos_idx[0][0]
                  predicted_seq = predicted_seq[:eos_idx]
                  targets_seq = targets_seq[:eos_idx]
        
        # reshape for cross-entropy loss
        output_flatten = output[1:].view(-1, output.shape[-1])
        trg_flatten = target_seq[1:].view(-1)

        loss = criterion(output_flatten, trg_flatten)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        predicted_seq=predicted_seq[1:].view(-1)
        targets_seq=targets_seq[1:].view(-1)
     
        correct_chars += torch.sum(predicted_seq == targets_seq).item()
        total_chars += targets_seq.numel()


    return total_loss / len(dataloader), correct_chars/total_chars, att_wt

In [ ]:

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0

    total_chars = 0
    correct_chars = 0

    with torch.no_grad():
        for i, (input_seq, target_seq) in enumerate(dataloader):
            input_seq = input_seq.long().to(device)
            target_seq = target_seq.long().to(device)

            output,att_wt = model(input_seq, target_seq,0)
            _, predicted = torch.max(output, dim=2)


            for j in range(predicted.shape[1]):
                predicted_seq = predicted[:, j]
                targets_seq = target_seq[:, j]

                # Find the index of the first EOS token in the sequence
                eos_idx = (targets_seq == hin_token_map["\n"]).nonzero()
                if eos_idx.numel() > 0:
                    eos_idx = eos_idx[0][0]
                    predicted_seq = predicted_seq[:eos_idx]
                    targets_seq = targets_seq[:eos_idx]


            # reshape for cross-entropy loss
            output_flatten = output[1:].view(-1, output.shape[-1])
            trg_flatten = target_seq[1:].view(-1)

            loss = criterion(output_flatten, trg_flatten)

            total_loss += loss.item()

            predicted_seq=predicted_seq[1:].view(-1)
            targets_seq=targets_seq[1:].view(-1)
     
            correct_chars += torch.sum(predicted_seq == targets_seq).item()
            total_chars += targets_seq.numel()

    return total_loss / len(dataloader), correct_chars/total_chars, att_wt

In [ ]:
'''
Best Model Configuration
Hidden unit-512
Embedding dim-128
Cell-LSTM
Lr-0.0001
Batch-64
Enc_dec_layer-1
Dropout-0.1
'''

In [ ]:
N_EPOCHS = 25

best_valid_loss = float('inf')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=custom_collate)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, collate_fn=custom_collate)

model=build_model(cell="LSTM",nunits=512,embed_dim=128,dropout=0.1)

model=model.to(device)
criterion = nn.CrossEntropyLoss(ignore_index=hin_token_map["<PAD>"]) # ignore padding index
optimizer = optim.Adam(model.parameters(), lr=1e-4)
for epoch in range(N_EPOCHS):    
    train_loss,acc,att_wt_train = train(model=model, dataloader=train_dataloader, optimizer=optimizer, criterion=criterion,device=device)
    valid_loss,ch,att_wt_val = evaluate(model, val_dataloader, criterion,device)
    

#torch.save(model.state_dict(), 'best_model_wth_attention.pt')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [ ]:
att_wt_val.shape

torch.Size([64, 1, 26])

In [ ]:
att_wt_val[1][0]


tensor([9.3250e-09, 5.6433e-08, 2.3335e-06, 1.2609e-04, 1.8822e-01, 7.0096e-01,
        9.0562e-02, 1.9813e-02, 3.0802e-04, 2.3399e-08, 3.5748e-11, 2.6407e-11,
        4.7198e-11, 7.5271e-11, 1.1243e-10, 1.6312e-10, 2.2970e-10, 3.1529e-10,
        4.2562e-10, 5.7033e-10, 7.6405e-10, 1.0272e-09, 1.3857e-09, 1.8664e-09,
        2.4781e-09, 3.1695e-09], device='cuda:0')

In [ ]:
%cp best_model_wth_attention.pt /content/drive/My\ Drive

In [ ]:
model.load_state_dict(torch.load('best_model_wth_attention.pt'))
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=custom_collate)
test_loss,test_acc = evaluate(model, test_dataloader, criterion,device)

In [ ]:
from matplotlib.font_manager import FontProperties
import matplotlib.ticker as ticker
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
mpl.rcParams['font.sans-serif'] = ["Arial Unicode MS"]

def plot_attention(attention, sentence, predicted_sentence,orig,hind,deco):
    
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(1, 1, 1)
    plott = ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}
    hindi_font = FontProperties(fname = 'VesperLibre-Regular.ttf')
    
    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontproperties=hindi_font, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
    
    title = f"Original word in english: {orig}\nOriginal word in hindi: {hind}\nDecoded word in hindi: {deco}"
    
    fontdict = {'fontsize': 18}
    plt.title(title,fontproperties=hindi_font, fontsize=14,y=-.3)
    
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    plt.colorbar(plott,cax=cax)
    
    plt.tight_layout()
    plt.savefig(f'res/{orig}.png', bbox_inches = 'tight')
    plt.show()

In [ ]:
real_hindi_list=[]
real_eng_list=[]
pred_hindi_list=[]


test_dataloader2 = DataLoader(test_dataset, batch_size=len(testx), shuffle=False, collate_fn=custom_collate)

input,target=next(iter(test_dataloader2))

total_correct=0
for i in range(len(input[0])-1):
  model.eval()
  with torch.no_grad():
      outputs = model(input[:,i:i+1].long().to(device), target[:,i:i+1].long().to(device), 0)
      output_idx = outputs[1:].squeeze(1).argmax(1)
  predicted_word=[]
  real_word_hindi=[]
  real_word_eng=[]

  for idx in output_idx.cpu():
    num=int(idx.numpy())
    if num ==2:
      break
    else:
      predicted_word.append(reverse_hin_map[num])

  for idx in target[:,i:i+1].numpy():
    if idx==2:
      break
    elif idx==1:
      pass
    else:
      real_word_hindi.append(reverse_hin_map[int(idx)])
  
  for idx in input[:,i:i+1].numpy():
    if idx==0:
      break
    elif idx==1:
      pass
    else:
      real_word_eng.append(reverse_eng_map[int(idx)])

  real_hindi_list.append("".join(real_word_hindi))
  real_eng_list.append("".join(real_word_eng))
  pred_hindi_list.append("".join(predicted_word))

  print("".join(predicted_word),"".join(real_word_hindi))
  if "".join(predicted_word)=="".join(real_word_hindi):
    total_correct+=1

#' '.join([target.vocab.itos[idx] for idx in output_idx])

print(f'Test Loss: {test_loss:.9f}')
print(f'Test Acc(charcter-level): {ch:.6f}')
print(f'Exact String Match: {total_correct/len(testy)}')

थर्मक्स थरमैक्स
सिखाएगा सिखाएगा
लीअर्न लर्न
ट्विटर्स ट्विटर्स
तिरुनेलवेली तिरुनेलवेली
इंडेपेंडेंस इंडिपेंडेंस
स्पेशियों स्पेशियों
शुरूह शुरूः
कोल्हापुर कोल्हापुर
अझर अजहर
करार क़रार
अंका अंक
वीपीडी डब्ल्यूपीडी
हाशी हाशिए
ग्लेंडले ग्लेंडल
उधेद उधेड़
एकथी इकठ्ठी
आईडीईए आईडिया
अम्बिकापुर अम्बिकापुर
मकेरेरे माकेरेरे
सबूदाने साबूदाने
फूहदतता फूहड़ता
सिक्वेंट सेक्वेंट
शुएब शुऐब
पनिहाटी पानीहाटी
समेतती समेटती
उखरुल उखरुल
ब्रा्मलिन ब्रह्मलीन
उतराधिकारी उतराधिकारी
इक़बाल इक़बाल
दयालपुरा दयालपुरा
सोहराई सोहराई
तकरीबन तक़रीबन
फर्रूखनगर फर्रूखनगर
थेंगा ठेंगा
त्योइहरों त्यौहारों
कर्नेश्वर्धाम कर्णेश्वरधाम
उमनाथ उमानाथ
दांशील दानशील
साहित्योत्सव साहित्योत्सव
शान्तिनिकेतन शांतिनिकेतन
शिकायतकर्ता शिकायतकर्ता
अंदरखाने अंदरखाने
पैंटर पंटर
लीदरों लीडरों
गलगंंड गलगंड
कार्नियाँ कार्नियां
मुर्गपालन मुर्गीपालन
मुशाहिद मुशाहिद
मोड्यूल्स मॉड्यूल्स
राजौरी रजौरी
सुश्रुषा सुश्रुषा
श्रृंगार शृंगार
होल्ट होल्ट
लैगिकता लैंगिकता
इजाजत इजाजत
वनक्षेत्र वनक्षेत्र
भुतल भूतल
स्वादप्रेमियों स्वादप्रेमियों
निनेतीज़ नाइनटीज


In [ ]:
pred_attention=pd.DataFrame({"English":real_eng_list,"Real_Hindi":real_hindi_list,"Predicted_Hindi":pred_hindi_list})

In [ ]:
pred_attention.to_csv("pred_attention.csv")

In [ ]:
pred_attention.to_csv("pred_attention.csv")